In [1]:
#using snake casing to name variables and functions

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import csv
import re

In [2]:
#global variables
city_name = "hyderabad"
zomato_url = "https://www.zomato.com"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}


In [3]:
def get_page_content(link,headers):
    try:
        #link = 'https://www.zomato.com/hyderabad/antera-kitchen-and-bar-jubilee-hills'
        response = requests.get(link, headers=headers)
        #response = requests.get("https://www.zomato.com/hyderabad/papadams-blue-kothapet/order")
        return {'status':response.status_code,
                'content': response.content}
    
    except requests.exceptions.HTTPError as errh:
        return {'status':response.status_code,
                'content':"Http Error:"+errh}
    except requests.exceptions.ConnectionError as errc:
        return {'status':response.status_code,
                'content':"Error Connecting:"+errc}
    except requests.exceptions.Timeout as errt:
        return {'status':response.status_code,
                'content':"Timeout Error:"+errt}
    except requests.exceptions.RequestException as err:
        return {'status':response.status_code,
                'content':"OOps: Something Else"+err}
    

In [4]:
def get_restaurant_links(city_name):
    try:
        url = "https://www.zomato.com/{}/restaurants".format(city_name)
        print(url)
        response = get_page_content(url,headers)
#         print(type(response))
        if(response['status']==200):
            content = response['content']
            soup = BeautifulSoup(content,"lxml")
            search_list = soup.find_all("div", {"class":"jumbo-tracker"})
            print("number of search_list:"+str(len(search_list)))

            restaurant_links=[]
            ################################ 
            #need to edit here, indices starts from 22 to len() ??? we need all!
            ################################ 
            for i in range(22,len(search_list)):
                link = search_list[i].div.find('a')['href']
                link = link.replace('/order', '')
                print(link)
                restaurant_links.append(zomato_url+link)

            print("total number of restaurant links:" + str(len(restaurant_links)))
            
            #     for x in (details[0].find_all(True)):
            #         if(x.string!=None):
            #             print(x.string+' {}'.format(i))
            #             i+=1
            e = {
                'status':200,
                'content':restaurant_links
            }
#             print(e)
            return e
    except Exception as e:
        e = {"status":999,'error':str(e)}
        print(e)
        return e

# print(get_restaurant_links(city_name))

In [5]:
# #just learnt abt beautiful-soup-4!

# jumbo_tracker = soup.select('.jumbo-tracker')
# tag = jumbo_tracker[0]
# #print tag-name
# print(tag.name) # yes its a div tag!
# #u can edit tag name in the current soup too!
# print(tag.attrs)
# print(tag.get('class'))
# print(type(soup))
# print(len(soup.find_all(attrs={'class':'jumbo-tracker'})))
# print(len(soup.find_all('div',attrs={'class':'jumbo-tracker'})))
# print(len(soup.select('div[class="jumbo-tracker"]')))
# print(soup.select('div[class="jumbo-tracker"]')[-1])
# print()

In [6]:
#returns get restaurant page content for 200 successful response

def get_primary_data(content):
    try:
        
        soup = BeautifulSoup(content,"lxml")

        main = soup.find("main").div

        # location = soup.find("div",{"aria-label":"Breadcrumb"}).find_all("a")
        location = main.section.div.find_all("a")
#         print(location)

        loc_tags = [tag["title"] for tag in location]

        rest_all_tags = {
            "country":loc_tags[1],
            "city":loc_tags[2],
            "city_2":loc_tags[3],
            "region":loc_tags[4],
            "restaurant_name":loc_tags[5],
        }
#         print(len(loc_tags))
#         print(loc_tags)
    #rest_all_tags["order_online"] = True if(loc_tags[6]=='Order Online') else False
    # order_online is invalid in off timings

    # loc_tags = loc_tags[1:len(loc_tags)-1]

#         print(rest_all_tags)
        
        e = {
            'status': 200,
            'content': rest_all_tags
        }
#         print(e)
        return e
    
    except Exception as e:
        e =  {
            'status': 998,
            'error': str(e)
        }
        print(e)
        return e

In [7]:
def get_secondary_data(content):
    try:
        soup = BeautifulSoup(content,"lxml")
        main = soup.find("main").div
        
        stuff = main.section.next_sibling.next_sibling.section
        stuff = stuff.get_text("!@#$%^&",strip=True).split("!@#$%^&")
#         print(stuff)
        
        rest_all_tags={}
        rest_all_tags["restaurant_name_2"] = stuff[0]
        rest_all_tags["dining_rating"] = stuff[1]
        rest_all_tags["dining_reviews"] = stuff[3]
        rest_all_tags["delivery_rating"] = stuff[5]
        rest_all_tags["delivery_reviews"] = stuff[7]
        rest_all_tags["cuisines"]=[stuff[9]]

        for i in range(9,len(stuff)):
            if(stuff[i-1]==','):
                rest_all_tags["cuisines"].append(stuff[i])

        cur = 9+(len(rest_all_tags["cuisines"]*2)-1)
        rest_all_tags["location"] = stuff[cur]

        #timings can't be captured in night or off timings!

#         print(len(stuff))
#         print(stuff)
        
#         print(rest_all_tags)
        e = {
            'status': 200,
            'content':rest_all_tags
        }
#         print(e)
        return e
    except Exception as e:
        e = {
            'status':997,
            'error':str(e)
        }
        print(e)
        return e
    

In [8]:
'''
for the rest of the info,
just get to the last but one sibling and find text,
then find its class then traverse into new one,
like cuisines: a,b,c
average_cost: a,b,c
'''

#
def get_more_info(content):
    try:
        soup = BeautifulSoup(content,"lxml")
        main = soup.find("main").div
        
        #another way for cuisines:
        
        # print(stuff.get_text('!@#$%^&').split("!@#$%^&"))
        # print(len(stuff.get_text('!@#$%^&').split("!@#$%^&")))

        search_elements = ['Cuisines', 'Popular Dishes', '₹','More Info']
        # print(stuff)
        stuff = main.section
        for i in range(4): #iterating through main.section siblings!
             stuff = stuff.next_sibling
        
        stuff = stuff.section.section.article.section.div.next_siblings

        cuisines = []
        popular_dishes = []
        average_costs = []
        more_info = []
        
        for q in stuff:
        #     print(q)
            if q.text == search_elements[0]:
                cuisines = q.next_sibling.get_text("!@#$%^&").split("!@#$%^&")
            elif q.text == search_elements[1]:
                popular_dishes = q.next_sibling.get_text("!@#$%^&").split("!@#$%^&")
            elif search_elements[2] in q.text:
                average_costs.append(q.text)
            elif q.text == search_elements[3]:
                more_info = q.next_sibling.get_text("!@#$%^&").split("!@#$%^&")


#         print(cuisines)
#         print(popular_dishes)
#         print(average_costs)
#         print(more_info)

        rest_all_tags={}
        rest_all_tags['cuisines_2'] = cuisines
        rest_all_tags['popular_dishes'] = popular_dishes
        rest_all_tags['average_costs']=average_costs
        rest_all_tags['more_info']=more_info
#         print(f"rest all tags:{rest_all_tags}")
        e = {
            'status': 200,
            'content':rest_all_tags
        }
#         print(e)
        return e
    except Exception as e:
        e = {
            'status': 996,
            'content':str(e)
        }
        print(e)
        return e

In [9]:
# df = pd.json_normalize(rest_all_tags) 
# print(df)

In [17]:
if __name__ == "__main__":
    try:
        r = get_restaurant_links(city_name)
        restaurant_links=[]
        if(r['status']!=200):
            r=get_restaurant_links(city_name)
        else:
            restaurant_links = r['content']
        
        columns = ['country','city','city_2','region','restaurant_name',
                   'restaurant_name_2','dining_rating', 
                   'dining_reviews', 'delivery_rating', 'delivery_reviews',
                   'cuisines','location', 'cuisines_2', 'popular_dishes',
                   'average_costs','more_info'
                  ]
        all_restaurant_details = pd.DataFrame(columns = columns)
        
        for link in restaurant_links:
            details={}
            page = get_page_content(link, headers)
    #         if(page['status'] != 200):
    #             continue
            page = page['content']
    #         print(link)
    #         print(details)

            data = get_primary_data(page)
            if(data['status']==200):
                details.update(data['content'])
            else:
                details.update({
                      'country': '',
                      'city': '',
                      'city_2': '',
                      'region': '',
                      'restaurant_name': ''
                })
    #         print(link)
    #         print(details)


            data = get_secondary_data(page)
            if(data['status']==200):
                details.update(data['content'])
            else:
                details.update({
                      'restaurant_name_2': '',
                      'dining_rating': 0.0,
                      'dining_reviews': 0,
                      'delivery_rating': 0.0,
                      'delivery_reviews': 0,
                      'cuisines': [],
                      'location': ''
                    })
    #         print(link)
    #         print(details)

            data = get_more_info(page)
            if(data['status']==200):
                details.update(data['content'])
            else:
                details.update({
                      'cuisines_2': [],
                      'popular_dishes': [],
                      'average_costs': [],
                      'more_info': []
                    })
                
#                 df = pd.DataFrame(technologies)
#                 new_row = {'Courses':'Hyperion', 'Fee':24000, 'Duration':'55days', 'Discount':1800}
#                 df2 = df.append(new_row, ignore_index=True)
                
            all_restaurant_details = all_restaurant_details.append(details, ignore_index=True)
    #         print(link)
    #         print(details)
            
    
    
    
    
#         df = pd.DataFrame(all_restaurant_details)
#         print(df)
        print(all_restaurant_details)
        print(len(all_restaurant_details))
        
        #to excel file
        all_restaurant_details.to_excel("{}_restaurants_details.xlsx".format(city_name), index=False)
        
    except Exception as e:
        print(e)
        print("error: "+str(e))
    
    
    
    
    
    

https://www.zomato.com/hyderabad/restaurants
number of search_list:31
/hyderabad/louis-burger-1-banjara-hills
/hyderabad/mithaiwala-kothapet
/hyderabad/amay-tiffin-corner-abids
/hyderabad/mehfil-narayanguda
/hyderabad/hotel-nayaab-charminar
/hyderabad/burger-king-malakpet
/hyderabad/peshawar-2-malakpet
/hyderabad/lucky-multicuisine-restaurant-somajiguda
/hyderabad/nimrah-restaurant-nampally
total number of restaurant links:9
  country       city          city_2         region  \
0   India  Hyderabad  Hyderabad City  Banjara Hills   
1   India  Hyderabad  Hyderabad City       Kothapet   
2   India  Hyderabad  Hyderabad City          Abids   
3   India  Hyderabad  Hyderabad City    Narayanguda   
4   India  Hyderabad  Hyderabad City      Charminar   
5   India  Hyderabad  Hyderabad City       Malakpet   
6   India  Hyderabad  Hyderabad City       Malakpet   
7   India  Hyderabad  Hyderabad City     Somajiguda   
8   India  Hyderabad  Hyderabad City       Nampally   

                    